In [ ]:
$ y = a + b x $

In this jupyter notebook, we will explain how to solve diffusion equation with the finite difference method. 
 To involve the physics, lets take heat diffusion equation for example:
        $$\rho C_p \frac {\partial T}  {\partial t} =\frac {\partial} {\partial x} (k \frac {\partial T} {\partial x})  $$

It can be simplified to: 
        $$ \frac {\partial T}  {\partial t} =\frac {\partial} {\partial x} (D \frac {\partial T} {\partial x})  $$

Now we can start to do the numerical discretization. One can do it directly on ths 2nd partial differential equation,but
here we will split it into two 1st order PDEs.  It is for the conveniance of introducing Pseudo-transient method with dampening. 
        $$ q =- D \frac {\partial T} {\partial x}  $$
        $$ \frac {\partial T}  {\partial t} =-\frac{\partial q} {\partial x}  $$

In fact, these two equations has physical meaning.q is the scaled heat flux ($q_{real}=q * \rho C_p$). 
Physically,q is the flux
$$ q_i =- D \frac {T_i-T_{i-1}} {\Delta x}  $$
    $$ \frac {{T_i}^{t+1}-{T_i}^t}  {\Delta t} =-\frac{{q_i}^t-{q_i}^t}  {\Delta x} \rightarrow \color {red} {\Delta T_{dt}}  $$
\

This leads to:
$${T_i}^{t+1}={T_i}^t+ \Delta t *\color {red}{\Delta T_{dt}}  $$


In [ ]:
It suggests that we can easily calculate the new T at any location if the old T field is known. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
Lx = 10
D = 100
nx = 150*1  # nx cell
dx = Lx / nx
Imax = 100 * nx

# Initiation
x = np.linspace(0, Lx, nx + 1)
xc = (x[:-1] + x[1:]) / 2
T0 = np.zeros(nx)
a = 0.5 * Lx
t0 = Lx * Lx / D / 1600
T0 = np.exp(-(xc - a) ** 4 / (4 * D * t0))
#T0 [xc<5]=1;
Told = T0
T = T0.copy()
f = np.sin(xc)
f[:] = 0

# Boundary condition
dTdt = np.zeros(nx - 2)

cnt = 100
epsi = 1e-3
tsc = Lx * Lx / D  # 100
CFL = 0.8
ndim = 1
itertol = 0
dtaudiff = CFL * dx * dx / (2 * D * ndim)
dt = dtaudiff * 0.01
time = 0
ttol = 0.1 * tsc  # 0.1 * tsc
Tana0 = np.sqrt(t0 / (t0 + ttol)) * np.exp(-(xc - a) ** 2 / (4 * D * (t0 + ttol)))
#T[0] = Tana0[0]
#T[-1] = Tana0[-1]

it = 0
plt.show()
while time < ttol * 0.999 and it < 10000:
    q = -D * np.diff(T) / dx
    dTdt = (-np.diff(q) / dx + f[1:-1])
    #T[1:-1] = T[1:-1] + dt * dTdt
    T[1:-1] = T[1:-1] + dt * dTdt

    it = it + 1
    time = time + dt
    Told = T
    if it % 100 == 0:
        #print(it)
        plt.clf()
        plt.plot(xc, T, 'b', xc, T0, 'k');


# Tana = 1 / np.sqrt(4 * (ttol + 1 / 4)) * np.exp(-(xc - a) ** 2 / (4 * D * (ttol + 1 / 4)))
Tana = np.sqrt(t0 / (t0 + time)) * np.exp(-(xc - a) ** 2 / (4 * D * (t0 + time)))
plt.figure()
plt.plot(xc, T, 'b', xc, Tana, 'r', xc, T0, 'k')
plt.show()

print('Total %d step are calculated.\nThe physical time is =%3.1f\n' % (it, time))


# play with  initial condition T,
# different dt. dt<()
# play with boundary condition
# change the code to solve the heat conduction problem: zero tempeature everywhere but high temperature at boundary


In [ ]:
print(Tana0[0])
print(np.size(T[1:-1]))